In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [4]:
path = "../training/conf/experiment/cnn_htr_char_lines.yaml"

In [5]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [6]:
cfg

{'defaults': [{'override /mapping': None}, {'override /criterion': None}, {'override /datamodule': None}, {'override /network': None}, {'override /model': None}, {'override /lr_schedulers': None}, {'override /optimizers': None}], 'criterion': {'_target_': 'torch.nn.CrossEntropyLoss', 'ignore_index': 3}, 'mapping': {'_target_': 'text_recognizer.data.emnist_mapping.EmnistMapping'}, 'optimizers': {'madgrad': {'_target_': 'madgrad.MADGRAD', 'lr': 0.0001, 'momentum': 0.9, 'weight_decay': 0, 'eps': 1e-06, 'parameters': 'network'}}, 'lr_schedulers': {'network': {'_target_': 'torch.optim.lr_scheduler.CosineAnnealingLR', 'T_max': 1024, 'eta_min': 4.5e-06, 'last_epoch': -1, 'interval': 'epoch', 'monitor': 'val/loss'}}, 'datamodule': {'_target_': 'text_recognizer.data.iam_lines.IAMLines', 'batch_size': 8, 'num_workers': 12, 'train_fraction': 0.8, 'augment': False, 'pin_memory': False}, 'network': {'_target_': 'text_recognizer.networks.conv_transformer.ConvTransformer', 'input_dims': [1, 56, 1024]

In [7]:
net = instantiate(cfg.network)

In [9]:
from torchinfo import summary

In [10]:
summary(net, ((4, 1, 56, 1024), (4, 89)), device="cpu")

torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 32, 1024])
torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 89, 1024])
torch.Size([4, 4, 32, 1024])


Layer (type:depth-idx)                             Output Shape              Param #
ConvTransformer                                    --                        --
├─EfficientNet: 1                                  --                        --
│    └─ModuleList: 2-1                             --                        --
├─Decoder: 1                                       --                        --
│    └─ModuleList: 2-2                             --                        --
│    │    └─ModuleList: 3-1                        --                        2,097,536
│    │    └─ModuleList: 3-2                        --                        2,097,536
│    │    └─ModuleList: 3-3                        --                        198,016
│    │    └─ModuleList: 3-4                        --                        2,097,536
│    │    └─ModuleList: 3-5                        --                        2,097,536
│    │    └─ModuleList: 3-6                        --                        198,0